In [1]:
import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F

import pandas as pd
import numpy as np

from utils.model import load_model
from utils.functionality import train, test, create_datasets
from collections import OrderedDict

In [ ]:
class Server():
    def __init__(self,clients: dict):
        self.globalmodel = load_model('Net')
        #for clients, there should be a nested dictionary
        self.clients = pd.DataFrame(clients)
        self.candidatesavg = {i+1: sum(self.clients.loc[i+1])/len(self.clients) for i in range(len(self.clients))}
        
    #### Vanilla FL Function    
    def aggregate(self,modelparams: list):
        params = [model.parameters() for model in modelparams] #obtain model params
        sumparams = [sum(param) for param in zip(*params)] #sum the model
        avgparams = [param/len(params) for param in sumparams] #average the model
        for param, avgparams in zip(self.globalmodel.parameters(), avgparams):
            param.data.copy_(avgparams.data)
            
    def sendparams(self,clientmodel):
        clientmodel.load_state_dict(self.globalmodel.state_dict())
        return clientmodel
    
    #### Proposed Idea Functions
    def choose_eval_cand(evalnum = 2, candidatesavg: dict, rounds):
        evaluator = []
        #### Select evaluators and early candidates
        for i in range(evalnum):
            tobeeval = max(candidatesavg,keys = candidatesavg.get)
            candidatesavg.pop(tobeeval,None)
            evaluator.append(tobeeval)
        #then the rest of the keys left becomes the candidates
        earlycandidates = []
        for key in candidatesavg.keys():
            earlycandidates.append(key)
            
        #### Assign probability to candidates
        candidatesprob = {}
        if rounds == 0:
            #client i model accuracy / all client model accuracy
            for i in earlycandidates:
                candidatesprob[i+1] = candidatesavg[i+1] / sum(candidatesavg.values())
        else:
            #how to simulate bid price?
        
        #### Select final candidates based on probability
        finalcandidates = []
        for i in candidatesprob.keys():
            if candidatesprob[i] > np.random.rand():
                finalcandidates.append(i)
            else:
                continue
        return evaluator, finalcandidates 
    
    def getclientinfo():
        pass
        return clients, modelparameters
            

In [ ]:
def main(numrounds):  
    #### initiate data
    local, test = create_datasets('./data','CIFAR10',num_clients=10, num_shards=200, iid=False, print_count = True)
    
    #### initiate federated learning rounds
    rounds = 0
    for i in range(numrounds):
        #client = Client()
        #parameters = client.parameter
        #identity = client.id
        #scores = client.scores
        
        #### Obtain weights
        client_params = {}
        for i in clients: #for models in clients
            client_params[client.id] = client.parameter
            
        #### Send weights
        for i in client_params: #for client id in saved client_id
            
        #### client send score back to server
        savedscore = {}
        #clients = {'cid1':{2:90, 3:40}, 
           #'cid2':{1:40,3:25}, 
           #'cid3':{1:65,2:86}}
        for i in clients: #for client id in saved client_id
            savedscore[i] = {}
            
        #clients, modelparams = getclientinfo() ###make this into 2 functions, one to get the score only, one to get the parameters
        
        #### Server helps choose candidate
        server = Server(clients, modelparams, evalnum, testdata)
        evaluator, finalcandidates = choose_eval_candidates(evalnum = 2, server.candidatesavg)
        #server sends weights of final candidates to evaluator. Evaluator test. Evaluator choose candidates who passed (criteria?)
        #evaluator sends clients who's accepted and declined
        #server only use clients who's accepted to be aggregated with the evaluator model
        #### Server aggregates      
        #what does the modelparams looks like? does it have client index? is it a list or an index?
        server.aggregate(modelparams) #aggregate
        
        loss, accuracy = test(server.globalmodel, test, device = "gpu")
        print("Round 1 metrics:")
        print("Loss = {}".format(loss))
        print("Accuracy = {}".format(accuracy))
        
        server.sendparams(clientmodel) #sending parameters
        
        #### Calculate contributions of clients 
        #after contributions is calculated, select next evaluator by contribution value
        #### give reputations 
        rounds += 1
        
        #### Begin round 2 by selecting candidates. Selection is different, be careful
        #repeat
        
        
        
    

In [ ]:
if __name__ == '__main__':
    main(10)

In [1]:
import pandas as pd

In [14]:
clientscores = {"eid0":{1:33, 2:23},"eid1":{0:46,2:70},"eid2":{0:90,1:22}}

def get_cand_avg(clientsscores):
        print("This is the client scores")
        print(clientsscores)
        clients = pd.DataFrame(clientsscores).fillna(0)
        #print("This is the Dataframe consist of model performance")
        display(clients)
        candidatesavg = {i: sum(clients.loc[i])/(len(clients)-1) for i in range(len(clients))}
        print("this is candidates avg!: {}".format(candidatesavg))
        return candidatesavg

First key is evaluator, second key is the client's performance on that evaluator data

In [15]:
clientscores

{'eid0': {1: 33, 2: 23}, 'eid1': {0: 46, 2: 70}, 'eid2': {0: 90, 1: 22}}

In [16]:
get_cand_avg(clientscores)

This is the client scores
{'eid0': {1: 33, 2: 23}, 'eid1': {0: 46, 2: 70}, 'eid2': {0: 90, 1: 22}}


,eid0,eid1,eid2
1,33.0,0.0,22.0
2,23.0,70.0,0.0
0,0.0,46.0,90.0


this is candidates avg!: {0: 68.0, 1: 27.5, 2: 46.5}


{0: 68.0, 1: 27.5, 2: 46.5}